In [143]:
import PIL as pillow
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import os
import pandas as pd
import itertools as it
#np.random.seed(123)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist



In [10]:
im=Image.open("Biwa Ko.jpg").convert("L")
imarr=np.array(im)
flatim=imarr.flatten('F')
flatim

array([46, 46, 46, ..., 27, 27, 27], dtype=uint8)

In [31]:
#!! pip install opencv-python

In [208]:
#from keras.datasets import mnist

#x = mnist.load_data()

In [209]:
#x#[1][0][1][1]
# this is the keras dataset

In [4]:
import cv2

In [30]:
img=cv2.imread("Biwa Ko.jpg")
img.shape

(400, 640, 3)

In [156]:
img=cv2.imread("Biwa Ko.jpg")
#img = img.reshape(img.shape[0], 400, 640, 3)
img.reshape(1,400,640,3)
img = img/255
img

array([[[0.20784314, 0.21176471, 0.10980392],
        [0.20784314, 0.21176471, 0.10980392],
        [0.20784314, 0.21176471, 0.10980392],
        ...,
        [0.18039216, 0.17647059, 0.07843137],
        [0.18039216, 0.17647059, 0.07843137],
        [0.18039216, 0.17647059, 0.07843137]],

       [[0.20784314, 0.21176471, 0.10980392],
        [0.20784314, 0.21176471, 0.10980392],
        [0.20784314, 0.21176471, 0.10980392],
        ...,
        [0.18039216, 0.17647059, 0.07843137],
        [0.18039216, 0.17647059, 0.07843137],
        [0.18039216, 0.17647059, 0.07843137]],

       [[0.20784314, 0.21176471, 0.10980392],
        [0.20784314, 0.21176471, 0.10980392],
        [0.20784314, 0.21176471, 0.10980392],
        ...,
        [0.18039216, 0.17647059, 0.07843137],
        [0.18039216, 0.17647059, 0.07843137],
        [0.18039216, 0.17647059, 0.07843137]],

       ...,

       [[0.17647059, 0.17254902, 0.07058824],
        [0.17647059, 0.17254902, 0.07058824],
        [0.17647059, 0

In [210]:

img.shape

(400, 640, 3)

In [129]:
# list_of_lakes = ['Biwa Ko.jpg', 'Black Lake.jpg']
list_of_img = []
list_of_names = []
path = '/Users/tovahirsch/Desktop/Capstone/Pics/'
for filename in os.listdir(path=path):
    if '.jpg' in filename:
        list_of_names.append(filename)
#         img=cv2.imread(path + filename, 0)
#         img = img/255
#         list_of_img.append(img)
#     break
# pd.DataFrame(list_of_names).to_csv('image_checklist.csv')

In [ ]:
def get_dict_from_csv():
    directory = os.listdir(path='/Users/tovahirsch/Desktop/Capstone/Pics_by_lat/')
    directory = [x for x in directory if '.jpg' in x]
    out = dict()
    for label in directory:
        out[label] = (df4.loc[df4['latitude']==label.replace('.jpg', ''), 'WQI']>44.00).astype(int)
    return out

In [ ]:
def image_gen(batch_size, y_dict):
    path = '/Users/tovahirsch/Desktop/Capstone/Pics_by_lat/'
    directory = os.listdir(path=path)
    Xout = []
    yout = []
    counter = 0
    for filename in it.cycle(directory):
        if '.jpg' in filename:
            img=cv2.imread(path + filename)
            img = img/255
            img.reshape(1,400,640,3)
            Xout.append(img)
            yout.append(y_dict[filename])
            counter += 1
            if counter % batch_size == 0:
                yield (np.array(Xout), np.array(yout))#np.random.randint(2, size=batch_size))
                Xout = []
                yout= []

In [196]:
gen = image_gen(32, get_dict_from_csv())

In [ ]:
type(gen)

In [204]:
next(gen)[0]

32

In [148]:
#set up model
model = Sequential()

model.add(Convolution2D(filters = 6, kernel_size = 3, activation='relu', input_shape=(400,640,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(16, kernel_size = 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [149]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 398, 638, 6)       168       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 199, 319, 6)       0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 197, 317, 16)      880       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 98, 158, 16)       0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 98, 158, 16)       0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 247744)            0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               31711360  
__________

In [151]:
# 6. Compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [96]:
img.shape

(400, 640, 3)

In [99]:
imgs = np.array([img])
imgs.shape

(1, 400, 640, 3)

In [177]:
#### 7. Fit model on training data
# model.fit(imgs,np.array([[0]]), batch_size=32, epochs=10, verbose=1)
model.fit_generator(image_gen(32), steps_per_epoch=1300//32)

Epoch 1/1
21/21 [==============================] - 137s 7s/step - loss: 2.5136 - acc: 0.4390


In [207]:
model.predict(np.array([img]))

array([[0.5201274]], dtype=float32)

In [11]:
# for x, y in zip(list_of_img, list_of_y):
#     model.fit(x, y, 
#           batch_size=32, epochs=10, verbose=1)

SyntaxError: unexpected EOF while parsing (<ipython-input-11-248255bab3d7>, line 1)

In [ ]:
'''cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)

    cv2.imshow('dst_rt', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    li.append(img)'''

In [ ]:
#img = cv2.resize(img, (800, 800))
cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
img

In [ ]:
cv2.imshow('dst_rt', img)
img

In [ ]:
cv2.waitKey(0)

In [ ]:
cv2.destroyAllWindows()

In [ ]:
img

In [51]:
img2=cv2.imread('Black Lake.jpg',0)
img2.shape

(400, 640)

# How do I use this array and transform it so that it has x and y which can then be pushed into the format as exist in mnist.load_data()

In [35]:
img.shape

(400, 640)

In [49]:
img[4].shape

(640,)

In [ ]:
plt.imshow(digits.images[-5], cmap=plt.cm.gray_r, interpolation='nearest')